In [1]:
import pandas as pd
import numpy as np

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler

from tensorflow.keras.models import load_model
import joblib


C:\Users\keert\SmartEnergyProject\venv\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [2]:
df = pd.read_csv("data/House_1_feature_engineered.csv")

if 'Time' in df.columns:
    df.drop(columns=['Time'], inplace=True)

y = df['Aggregate']
X = df.drop(columns=['Aggregate'])

X.shape, y.shape


((269635, 22), (269635,))

In [3]:
X_scaler = MinMaxScaler()
y_scaler = MinMaxScaler()

X_scaled = X_scaler.fit_transform(X)
y_scaled = y_scaler.fit_transform(y.values.reshape(-1, 1))


In [5]:
train_size = int(len(X_scaled) * 0.7)
val_size = int(len(X_scaled) * 0.15)

X_test = X_scaled[train_size + val_size:]
y_test = y_scaled[train_size + val_size:]

def create_sequences(X, y, lookback=24):
    Xs, ys = [], []
    for i in range(len(X) - lookback):
        Xs.append(X[i:i+lookback])
        ys.append(y[i+lookback])
    return np.array(Xs), np.array(ys)

lookback = 24
X_test_seq, y_test_seq = create_sequences(X_test, y_test, lookback)

X_test_seq.shape, y_test_seq.shape


((40422, 24, 22), (40422, 1))

In [6]:
lstm_model = load_model("lstm_energy_model.keras")


In [7]:
y_test_pred = lstm_model.predict(X_test_seq)

lstm_mae = mean_absolute_error(y_test_seq, y_test_pred)
lstm_rmse = np.sqrt(mean_squared_error(y_test_seq, y_test_pred))
lstm_r2 = r2_score(y_test_seq, y_test_pred)

lstm_mae, lstm_rmse, lstm_r2


1264/1264 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step


(0.008000906073022325, np.float64(0.02426234228174005), 0.422449805808929)

In [8]:
baseline_mae = 130.1952214296498 
baseline_rmse = 408.13082619298825
baseline_r2=0.738110113598123

In [9]:
evaluation = pd.DataFrame({
    'Model': ['Linear Regression (Baseline)', 'LSTM'],
    'MAE': [baseline_mae, lstm_mae],
    'RMSE': [baseline_rmse, lstm_rmse],
    'R2 Score': [baseline_r2, lstm_r2]
})

evaluation


,Model,MAE,RMSE,R2 Score
0,Linear Regression (Baseline),130.195221,408.130826,0.73811
1,LSTM,0.008001,0.024262,0.42245


In [10]:
lstm_model.save("lstm_energy_model.keras")
joblib.dump(y_scaler, "target_scaler.pkl")


['target_scaler.pkl']

In [11]:
def predict_energy_lstm(input_sequence):
    scaled_pred = lstm_model.predict(input_sequence)
    real_pred = y_scaler.inverse_transform(scaled_pred)
    return float(real_pred[0][0])


In [ ]:
sample_input = X_test_seq[0].reshape(1, 24, X_test_seq.shape[2])

predicted = predict_energy_lstm(sample_input)
actual = y_scaler.inverse_transform(y_test_seq[0].reshape(1,1))[0][0]

actual, predicted
